<a href="https://colab.research.google.com/github/andygoosh/samsung/blob/master/Arxiv_title_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from pathlib import Path
from google.colab import files, drive
import pandas as pd
import zipfile
from io import BytesIO
from urllib.request import urlopen
import tensorflow as tf

gpath = Path('/content/gdrive')
drive.mount(str(gpath))
data_file = gpath / 'My Drive/Samsung' / 'train.csv.zip'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [20]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cpu':
    print('cpu')
else:
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0))

Tesla T4


In [18]:
train_df = pd.read_csv(data_file)
train_df

,abstract,title
0,we consider the problem of utility maximizatio...,on optimal investment with processes of long o...
1,in this paper we provide an explicit formula f...,boolean complexes for ferrers graphs
2,"kinesin-5, also known as eg5 in vertebrates is...",relative velocity of sliding of microtubules b...
3,we discuss the transition paths in a coupled b...,bifurcation of transition paths induced by cou...
4,two types of room temperature detectors of ter...,all-electric detectors of the polarization sta...
...,...,...
134995,we consider a minimalistic dynamic model of th...,randomly evolving idiotypic networks: structur...
134996,this is an extended version of a communication...,noncommutative generalization of su(n)-princip...
134997,"in this paper, a new method was developed for ...",initialization of multilayer forecasting artif...
134998,we propose a new approach to analyze data that...,principal arc analysis on direct product manif...
